# Hierarchical Forecasting

In this notebook we take the example of demand forecasting on synthetic retail data and show you how to train and tune multiple hierarchichal time series models across algorithms and hyper-parameter combinations using the `scikit-hts` toolkit on Amazon SageMaker. We will first show you how to setup scikit-hts on SageMaker using the SKLearn estimator, then train multiple models using SageMaker Experiments, and finally use SageMaker Debugger to monitor suboptimal training and improve training efficiencies. We will walk you through the following steps:

1.	[Setup](#Setup)
2.	[Prepare Time Series Data](#Prepare-Time-Series-Data)
    - [Data Visualization](#Data-Visualization)
    - [Split data into train and test](#Split-data-into-train-and-test)
    - [Hierarchical Representation](#Hierarchical-Representation)
    - [Visualizing the tree structure](#Visualizing-the-tree-structure)
3.	[Setup the scikit-hts training script](#section3)
4.  [Setup Amazon SageMaker Experiment and Trials](#section4)
5.	[Setup the SKLearn Estimator](#section5)
6.	[Evaluate metrics and select a winning candidate](#section7)
7.	[Run time series forecasts](#Run-time-series-forecasts)
    - [Visualization at Region Level](#Visualization-at-Region-Level)
    - [Visualization at State Level](#Visualization-at-State-Level)


Before getting started we need to first install a few packages:

## Setup 

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
! pip install upgrade pip

In [ ]:
!pip install --user scikit-hts[prophet]

In [ ]:
#! pip install --user plotly -q 
! pip install --user scikit-hts -q 
! pip install --user -U sagemaker -q

In [ ]:
!conda install -c plotly plotly --yes
!conda install -c conda-forge fbprophet --yes

**Important** -- Make sure to restart the kernel.

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from hts.hierarchy import HierarchyTree
from hts import HTSRegressor
import numpy as np

In [ ]:
from pandas import DataFrame
from pandas import Grouper
from matplotlib import pyplot
import pandas as pd
import dataset_prep

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.CRITICAL)

In [ ]:
import boto3
import sagemaker 

s3_client = boto3.client('s3')
s3res = boto3.resource('s3')

sess = sagemaker.Session()
bucket = sess.default_bucket()

pref = 'hierarchical-forecast-retail/scikit-hts'
s3_train_channel = "s3://" + bucket + "/" + pref + "/train.csv"
s3_test_channel = "s3://" + bucket + "/" + pref + "/test.csv"
print(s3_train_channel)
print(s3_test_channel)

## Prepare Time Series Data

In [ ]:
## Read cleaned, joined, featurized data.
df_raw = pd.read_csv("retail-usa-clothing.csv"
                          , parse_dates=True
                          , header=0
                          , names=['date', 'state'
                                   , 'item', 'quantity', 'region'
                                   , 'country']
                    )
df_raw['quantity'] = df_raw['quantity'].astype(int)
# drop duplicates
print(df_raw.shape)
df_raw.drop_duplicates(inplace=True)

df_raw['date'] = pd.to_datetime(df_raw["date"])
print(df_raw.shape)
print(df_raw.dtypes)
print(f"Min timestamp = {df_raw.date.min()}")
print(f"Max timestamp = {df_raw.date.max()}")
df_raw.sample(5)

In [ ]:
df_raw.region.unique()

In [ ]:
df_raw.head()

In [ ]:
## # map expected column names
item_id = "item"
target_value = "quantity"
timestamp = "date"
city = "city"
region = 'region'
country = 'country'

### Drop null item_ids

In [ ]:
## Drop null item_ids
templist = df_raw[item_id].unique()
print(f"Number unique items: {len(templist)}")
print(f"Number nulls: {pd.isnull(templist).sum()}")

if len(templist) < 20:
    print(templist)

In [ ]:
## Drop the null item_ids, if any exist
if pd.isnull(templist).sum() > 0:
    print(df_raw.shape)
    df_raw = df_raw.loc[(~df_raw[item_id].isna()), :].copy()
    print(df_raw.shape)
    print(len(df_raw[item_id].unique()))
else:
    print("No missing item_ids found.")

### Drop null timestamps

In [ ]:
# check null timestamps
templist = df_raw.loc[(df_raw[timestamp].isna()), :].shape[0]
print(f"Number nulls: {templist}")

if (templist < 10) & (templist > 0) :
    print(df_raw.loc[(df_raw[timestamp].isna()), :])

In [ ]:
## Drop the null quantities and dates
if templist > 0:
    print(df_raw.shape)
    df_raw = df_raw.loc[(~df_raw[timestamp].isna()), :].copy()
    print(df_raw.shape)
    print(df_raw['date'].isna().sum())
else:
    print("No null dates found.")

In [ ]:
from pandas import DataFrame
from pandas import Grouper
from matplotlib import pyplot
groups = df_raw.groupby(Grouper(key=item_id))

## Data Visualization

In [ ]:
## Plots
for name, group in groups:
    group.plot(subplots=True,title=name, x=timestamp, y=target_value, legend=True)
    pyplot.show()

### Split data into train and test

In [ ]:
df_train = df_raw.query(f'date <= "2009-04-29"').copy()
df_train.to_csv("train.csv")
s3_client.upload_file("train.csv", bucket, pref+"/train.csv")

In [ ]:
df_test = df_raw.query(f'date > "2009-04-29"').copy()
df_test.to_csv("test.csv")
s3_client.upload_file("test.csv", bucket, pref+"/test.csv")

In [ ]:
df_train.head()

In [ ]:
product = df_train[(df_train['item'] == "mens_clothing")]
product.head()

In [ ]:
def get_region_columns(df, region):
    return [col for col in df.columns if region in col]


In [ ]:
product["region_state"] = product.apply(lambda x: f"{x['region']}_{x['state']}", axis=1)
region_states = product["region_state"].unique()
grouped_sections = product.groupby(["region", "region_state"])
edges_hierarchy = list(grouped_sections.groups.keys())
# Now, we must not forget that total is our root node.
second_level_nodes = product.region.unique()
root_node = "total"
root_edges = [(root_node, second_level_node) for second_level_node in second_level_nodes]
root_edges += edges_hierarchy
product_bottom_level = product.pivot(index="date", columns="region_state", values="quantity")
regions = product["region"].unique().tolist()
for region in regions:
    region_cols = get_region_columns(product_bottom_level, region)
    product_bottom_level[region] = product_bottom_level[region_cols].sum(axis=1)

product_bottom_level["total"] = product_bottom_level[regions].sum(axis=1)

# create hierarchy
# Now that we have our dataset ready, let's define our hierarchy tree. 
# We need a dictionary, where each key is a column (node) in our hierarchy and a list of its children.
hierarchy = dict()

for edge in root_edges:
    parent, children = edge[0], edge[1]
    hierarchy.get(parent)
    if not hierarchy.get(parent):
        hierarchy[parent] = [children]
    else:
        hierarchy[parent] += [children]

product_bottom_level.index = pd.to_datetime(product_bottom_level.index)
product_bottom_level = product_bottom_level.resample("D").sum()

## Hierarchical Representation

*scikit-hts* requires that each column in our DataFrame is a time series of its own, for all hierarchy levels. Let's do that. Remember that our data is in a long format.

The steps are the following:

1. Transform dataset into a column oriented one
2. Create the hierarchy representation as a dictionary
 
For a complete description of how that is done under the hood, and for a sense of what the API accepts, see [scikit-hts' docs](https://scikit-hts.readthedocs.io/en/latest/hierarchy.html)

In [ ]:
#Let's take a look at the training script
!pygmentize code/dataset_prep.py

In [ ]:
train_hierarchy, train_product_bottom_level, region_states = dataset_prep.prepare_data(df_train)

In [ ]:
test_hierarchy, test_product_bottom_level, region_states = dataset_prep.prepare_data(df_test)

## Visualizing the tree structure

In [ ]:
from hts.hierarchy import HierarchyTree

ht = HierarchyTree.from_nodes(nodes=train_hierarchy, df=train_product_bottom_level)

In [ ]:
print(ht)

In [ ]:
print(ht.children[1].key)
print(ht.children[1])

In [ ]:
print(ht.get_node('NewEngland'))

In [ ]:
regions = df_raw["region"].unique().tolist()
regions

## Create the algorithm and hyper-parameters combinatorial matrix <a name=section2></a>

In [ ]:
import pandas as pd
d = {'revision_method': ["BU", "AHP"], 'seasonality_mode':["additive", "multiplicative"]}
df_hps = pd.DataFrame(data=d)
df_hps.head()

We will use the 'product' function to derive combinations of these parameters from the base set into separate rows in the dataframe. Each row corresponds to a training job configuration that we will subsequently pass to the SKLearn Estimator to run the training job.

Note Please check your AWS account limits before you setup the product function below. The training process in the sections below will run one training job per row from this dataframe. Based on your account limit for the maximum number of concurrent training jobs, you may get an error that the limit has been exceeded.

In [ ]:
from itertools import product

prod = product(df_hps['seasonality_mode'].unique(), df_hps['revision_method'].unique())

df_hps_combo = pd.DataFrame([list(p) for p in prod],
                   columns=list(['seasonality_mode', 'revision_method']))

df_hps_combo['jobnumber'] = df_hps_combo.index

Let's take a look on the different combinations. 

In [ ]:
df_hps_combo

## Setup the scikit-hts training script

In [ ]:
#Let's take a look at the training script
!pygmentize code/train.py

## Setup a SageMaker Experiment  <a name=section4></a>

Before create the training job, we first create a SageMaker Experiment that will allow us to track the different training jobs. We use the `smexperiments` libraray to create the experiment:

In [ ]:
from datetime import datetime
from smexperiments.experiment import Experiment

sagemaker_boto_client = boto3.client("sagemaker")

#name of experiment
timestep = datetime.now()
timestep = timestep.strftime("%d-%m-%Y-%H-%M-%S")
experiment_name = "hierarchical-forecast-models-" + timestep

#create experiment
Experiment.create(
    experiment_name=experiment_name, 
    description="Hierarchical Timeseries models", 
    sagemaker_boto_client=sagemaker_boto_client)

For each job we define a new Trial component within that experiment:

In [ ]:
from smexperiments.trial import Trial

trial = Trial.create(
    experiment_name=experiment_name,
    sagemaker_boto_client=sagemaker_boto_client
)
print(trial)

In [ ]:
experiment_config = { "ExperimentName": experiment_name, 
                      "TrialName":  trial.trial_name,
                      "TrialComponentDisplayName": "Training"}

## Fitting models

We will use scikit-hts to fit `Prophet` model in our data and compare results.
- Prophet: 
    - *daily_seasonality* : By default daily seasonality is set to `False`, therefore, explicitly changing it to `True`
    - *changepoint_prior_scale* : If the trend changes are being overfit (too much flexibility) or underfit (not enough flexibility), you can adjust the strength of the sparse prior using the input argument changepoint_prior_scale. By default, this parameter is set to `0.05`. Increasing it will make the trend more flexible.


In [ ]:
metric_definitions=[
    {'Name': 'Total:MSE', 'Regex': 'Total: MSE: ([0-9\\.]+)'},
    {'Name': 'Mid-Alantic:MSE', 'Regex': 'Mid-Alantic: MSE: ([0-9\\.]+)'},
    {'Name': 'SouthCentral:MSE', 'Regex': 'SouthCentral: MSE: ([0-9\\.]+)'},
    {'Name': 'Pacific:MSE', 'Regex': 'Pacific: MSE: ([0-9\\.]+)'},
    {'Name': 'EastNorthCentral:MSE', 'Regex': 'EastNorthCentral: MSE: ([0-9\\.]+)'},
    {'Name': 'NewEngland:MSE', 'Regex': 'NewEngland: MSE: ([0-9\\.]+)'},
    {'Name': 'NewYork:MSE', 'Regex': 'NewYork:MSE: ([0-9\\.]+)'},
    {'Name': 'Alabama:MSE', 'Regex': 'Alabama:MSE: ([0-9\\.]+)'},
    {'Name': 'Alaska:MSE', 'Regex': 'Alaska:MSE: ([0-9\\.]+)'},
    {'Name': 'Kentucky:MSE', 'Regex': 'Kentucky:MSE: ([0-9\\.]+)'},
    {'Name': 'Illinois:MSE', 'Regex': 'Illinois:MSE: ([0-9\\.]+)'},
    {'Name': 'Mississippi:MSE', 'Regex': 'Mississippi:MSE: ([0-9\\.]+)'},
    {'Name': 'Hawaii:MSE', 'Regex': 'Hawaii:MSE: ([0-9\\.]+)'},
    {'Name': 'Indiana:MSE', 'Regex': 'Indiana:MSE: ([0-9\\.]+)'},
    {'Name': 'NewJersey:MSE', 'Regex': 'NewJersey:MSE: ([0-9\\.]+)'},
    {'Name': 'Pennsylvania:MSE', 'Regex': 'Pennsylvania:MSE: ([0-9\\.]+)'},
    {'Name': 'Tennessee:MSE', 'Regex': 'Tennessee:MSE: ([0-9\\.]+)'},
    {'Name': 'California:MSE', 'Regex': 'California:MSE: ([0-9\\.]+)'},
    {'Name': 'RhodeIsland:MSE', 'Regex': 'RhodeIsland:MSE: ([0-9\\.]+)'},
    {'Name': 'Oregon:MSE', 'Regex': 'Oregon:MSE: ([0-9\\.]+)'},
    {'Name': 'Connecticut:MSE', 'Regex': 'Connecticut:MSE: ([0-9\\.]+)'},
    {'Name': 'Maine:MSE', 'Regex': 'Maine:MSE: ([0-9\\.]+)'},
    {'Name': 'Ohio:MSE', 'Regex': 'Ohio:MSE: ([0-9\\.]+)'},
    {'Name': 'Vermont:MSE', 'Regex': 'Vermont:MSE: ([0-9\\.]+)'},
]

### Create the SKLearn Estimator  <a name=section5></a>


In [ ]:
import sagemaker
from sagemaker.sklearn import SKLearn

for idx, row in df_hps_combo.iterrows():
    trial = Trial.create(
        experiment_name=experiment_name,
        sagemaker_boto_client=sagemaker_boto_client
    )

    experiment_config = { "ExperimentName": experiment_name, 
                      "TrialName":  trial.trial_name,
                      "TrialComponentDisplayName": "Training"}
    

    sklearn_estimator = SKLearn('train.py',
                                source_dir='code',
                                instance_type='ml.m4.xlarge',
                                framework_version='0.23-1',
                                role=sagemaker.get_execution_role(),
                                debugger_hook_config=False,
                                hyperparameters = {'bucket': bucket,
                                                   'algo': "Prophet", 
                                                   'daily_seasonality': True,
                                                   'changepoint_prior_scale': 0.5,
                                                   'seasonality_mode': row['seasonality_mode'],
                                                   'revision_method' : row['revision_method']
                                                  },
                                metric_definitions = metric_definitions,
                               )
    sklearn_estimator.fit({'train': s3_train_channel, "test": s3_test_channel},
                     experiment_config=experiment_config, wait=True)

Once the experiment is finished we can determine how many seconds it ran. First we define a helper function to compute the billabale seconds and how many training jobs were auto-terminated.

## Evaluate metrics and select a winning candidate <a name=section8></a>
Amazon SageMaker Studio provides an experiments browser that you can use to view lists of experiments, trials, and trial components. You can choose one of these entities to view detailed information about the entity or choose multiple entities for comparison. For more details please refer to [the documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments-view-compare.html#experiments-view). Once the training jobs are running we can use the experiment view in Studio (see screenshot below) or the `ExperimentAnalytics` module to track the status of our training jobs and their metrics. 
![](screenshot.png)


In the training script we used SageMaker Debugger's function `save_scalar` to store metrics such as MAPE, MSE, RMSE in the experiment. We can access the recorded metrics via the ExperimentAnalytics function and convert it to a Pandas dataframe.


In [ ]:
def compute_job_statistics(df):
    total_cost  = 0
    stopped = 0
    for name in df['sagemaker_job_name']:
        description = sagemaker_boto_client.describe_training_job(TrainingJobName=name[1:-1])
        total_cost += description['BillableTimeInSeconds']
        if description['TrainingJobStatus'] == "Stopped":
            stopped += 1
    return stopped, total_cost

In [ ]:
from sagemaker.analytics import ExperimentAnalytics
trial_component_analytics = ExperimentAnalytics(experiment_name=experiment_name)

stopped, total_cost = compute_job_statistics(trial_component_analytics.dataframe())
print("Billable seconds for overall experiment:", total_cost, "seconds. Number of training jobs auto-terminated:", stopped)

This setup is especially useful if you run a parameter sweep with training jobs that train for hours. In our case each job only trained for less than 10 minutes. Until the Debugger data is uploaded, fetched and downloaded into the processing job, a few minutes may pass, so the potential cost reduction will be less for smaller training jobs.

In [ ]:
from sagemaker.analytics import ExperimentAnalytics

trial_component_analytics = ExperimentAnalytics(experiment_name=experiment_name)
tc_df = trial_component_analytics.dataframe()
tc_df

In [ ]:
total_mse = []
model_url = []
for name in tc_df['sagemaker_job_name']:
        description = sagemaker_boto_client.describe_training_job(TrainingJobName=name[1:-1])
        total_mse.append(description['FinalMetricDataList'][0]['Value'])
        model_url.append(description['ModelArtifacts']['S3ModelArtifacts'])
tc_df['total_mse'] = total_mse

In [ ]:
description['FinalMetricDataList'][0]['Value']

Let's take a look on the metrics and hyperparameter combinations:

In [ ]:
new_df = tc_df[['sagemaker_job_name','algo', 'changepoint_prior_scale', 'revision_method', 'total_mse', 'seasonality_mode']]
new_df  

In [ ]:
mse_min = new_df['total_mse'].min()
mse_min

Let's select the winner model:

In [ ]:
df_winner = new_df[new_df['total_mse'] == mse_min]

Download the winning model for running forecasts.

In [ ]:
for name in df_winner['sagemaker_job_name']:
    model_dir = sagemaker_boto_client.describe_training_job(TrainingJobName = name[1:-1])['ModelArtifacts']['S3ModelArtifacts']
model_dir

In [ ]:
model_dir

In [ ]:
key = model_dir.split('s3://{}/'.format(bucket))

In [ ]:
s3_client.download_file(bucket, key[1], 'model.tar.gz')

In [ ]:
!tar -xvzf model.tar.gz

## Run time series forecasts
First, let's load the model. 

In [ ]:
import joblib
def model_fn(model_dir):
    clf = joblib.load(model_dir)
    return clf

In [ ]:
model = model_fn('model.joblib')

In [ ]:
model

Now, let's make forecasts 90 days in future.

In [ ]:
predictions = model.predict(steps_ahead=90)

Let's visualize the model results and fitted values for all states.

In [ ]:
## Helper function for plotting results.
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
def plot_results(cols, axes, preds):
    axes = np.hstack(axes)
    for ax, col in zip(axes, cols):
        preds[col].plot(ax=ax, label="Predicted")
        train_product_bottom_level[col].plot(ax=ax, label="Observed")
        ax.legend()
        ax.set_title(col)
        ax.set_xlabel("Date")
        ax.set_ylabel("Quantity")    

#### Visualization at Region Level

In [ ]:
fig, axes = plt.subplots(len(regions), 1, figsize=(10, 20))
plot_results(regions, axes, predictions)
plt.tight_layout()

#### Visualization at State Level

In [ ]:
fig, axes = plt.subplots(len(region_states), 1, figsize=(20, 70))
plot_results(region_states, axes, predictions)
plt.tight_layout()